<a href="https://colab.research.google.com/github/manmeetks/comparison-of-text-sentence-similarity-models-using-TOPSIS/blob/main/implementation_of_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
df=pd.read_csv("clean.csv")

In [55]:
df.describe()

,same_security,ticker_x_AA,ticker_x_AAGPX,ticker_x_AAPL,ticker_x_ABC,ticker_x_ABEYX,ticker_x_ABT,ticker_x_ACGMX,ticker_x_ACN,ticker_x_ADP,...,ticker_y_ZNGA,ticker_y_ZOES,cosine_similarity_doc2vec,cosine_similarity_use,jaccard_similarity,cosine_similarity_tfidf,cosine_similarity_w2v,1,0,true_labels
count,2142.000000,2142.000000,2142.000000,2142.000000,2142.000000,2142.000000,2142.000000,2142.000000,2142.000000,2142.000000,...,2142.000000,2142.000000,2142.000000,2142.000000,2142.000000,2.142000e+03,0.0,2142.0,2142.0,2142.000000
mean,0.753035,0.000467,0.001401,0.003735,0.000467,0.000467,0.000934,0.000934,0.002334,0.000467,...,0.001867,0.000934,0.721744,0.830546,0.402078,1.000000e+00,NaN,1.0,0.0,0.753035
std,0.431347,0.021607,0.037407,0.061013,0.021607,0.021607,0.030549,0.030549,0.048269,0.021607,...,0.043183,0.030549,0.539580,0.124715,0.226414,1.635642e-16,NaN,0.0,0.0,0.431347
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-0.989814,0.364172,0.000000,1.000000e+00,NaN,1.0,0.0,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.841782,0.759651,0.230769,1.000000e+00,NaN,1.0,0.0,1.000000
50%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.972926,0.857182,0.400000,1.000000e+00,NaN,1.0,0.0,1.000000
75%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.989131,0.929965,0.571429,1.000000e+00,NaN,1.0,0.0,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,0.999986,1.000000,1.000000,1.000000e+00,NaN,1.0,0.0,1.000000


In [3]:
df.head()

,description_x,description_y,same_security,ticker_x_AA,ticker_x_AAGPX,ticker_x_AAPL,ticker_x_ABC,ticker_x_ABEYX,ticker_x_ABT,ticker_x_ACGMX,...,ticker_y_XOM,ticker_y_XONE,ticker_y_XT,ticker_y_YACKX,ticker_y_YHOO,ticker_y_YUM,ticker_y_Z,ticker_y_ZBH,ticker_y_ZNGA,ticker_y_ZOES
0,first trust dow jones internet,first trust dj internet idx,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,schwab intl large company index etf,schwab strategic tr fundamental intl large co ...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,vanguard small cap index adm,vanguard smallcap index fund inst,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,duke energy corp new com new isin us4 sedol b7...,duke energy corp new com new isin us26441c2044...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,visa inc class,visa inc,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
df.replace([np.nan, np.inf, -np.inf], 0, inplace=True)


In [5]:
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.5 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=c153d97aace447d79c5f22f4cc13cc09f25f63f5db7f90229a44c06db850926c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sentence_transformers import SentenceTransformer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error


In [7]:
# Function to calculate cosine similarity
def calculate_cosine_similarity(vec1, vec2):
    return cosine_similarity([vec1], [vec2])[0][0]


In [20]:
w2v_model = Word2Vec(sentences=df['description_x'].apply(word_tokenize), vector_size=100, window=5, min_count=1, workers=4)

def calculate_cosine_similarity_safe(vec1, vec2, model):
    # Check if both words are present in the vocabulary
    if vec1 in model.wv.key_to_index and vec2 in model.wv.key_to_index:
        return cosine_similarity([model.wv[vec1]], [model.wv[vec2]])[0][0]
    else:
        return np.nan  # or any value that indicates missing similarity

df['cosine_similarity_w2v'] = df.apply(lambda row: calculate_cosine_similarity_safe(row['description_x'], row['description_y'], w2v_model), axis=1)


In [58]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error
true_labels = df['same_security']

threshold = 0.5
binary_preds_w2v = (df['cosine_similarity_w2v'] > 0.5).astype(int)

# Calculate evaluation metrics
accuracy_w2v = accuracy_score(true_labels, binary_preds_w2v)
precision_w2v = precision_score(true_labels, binary_preds_w2v)
recall_w2v = recall_score(true_labels, binary_preds_w2v)
f1_w2v = f1_score(true_labels, binary_preds_w2v)
mse_w2v = mean_squared_error(true_labels, df['cosine_similarity_w2v'])

print("Word2Vec Model Metrics:")
print(f"Accuracy: {accuracy_w2v:.2f}")
print(f"Precision: {precision_w2v:.2f}")
print(f"Recall: {recall_w2v:.2f}")
print(f"F1 Score: {f1_w2v:.2f}")
print(f"Mean Squared Error: {mse_w2v:.2f}")


Word2Vec Model Metrics:
Accuracy: 0.25
Precision: 0.00
Recall: 0.00
F1 Score: 0.00
Mean Squared Error: 0.75


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
# Doc2Vec
tagged_data = [TaggedDocument(words=word_tokenize(text), tags=[str(i)]) for i, text in enumerate(df['description_x'])]
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=10)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
df['cosine_similarity_doc2vec'] = df.apply(lambda row: calculate_cosine_similarity(doc2vec_model.infer_vector(word_tokenize(row['description_x'])), doc2vec_model.infer_vector(word_tokenize(row['description_y']))), axis=1)


In [59]:
true_labels = df['same_security']

threshold = 0.5
binary_preds_doc2vec = (df['cosine_similarity_doc2vec'] > threshold).astype(int)

# Calculate evaluation metrics for Doc2Vec model
accuracy_doc2vec = accuracy_score(true_labels, binary_preds_doc2vec)
precision_doc2vec = precision_score(true_labels, binary_preds_doc2vec)
recall_doc2vec = recall_score(true_labels, binary_preds_doc2vec)
f1_doc2vec = f1_score(true_labels, binary_preds_doc2vec)
mse_doc2vec = mean_squared_error(true_labels, df['cosine_similarity_doc2vec'])

print("Doc2Vec Model Metrics:")
print(f"Accuracy: {accuracy_doc2vec:.2f}")
print(f"Precision: {precision_doc2vec:.2f}")
print(f"Recall: {recall_doc2vec:.2f}")
print(f"F1 Score: {f1_doc2vec:.2f}")
print(f"Mean Squared Error: {mse_doc2vec:.2f}")


Doc2Vec Model Metrics:
Accuracy: 0.76
Precision: 0.81
Recall: 0.89
F1 Score: 0.85
Mean Squared Error: 0.35


In [12]:
# Universal Sentence Encoder (USE)
use_model = SentenceTransformer('bert-base-nli-mean-tokens')
embeddings_x = use_model.encode(df['description_x'])
embeddings_y = use_model.encode(df['description_y'])
df['cosine_similarity_use'] = [calculate_cosine_similarity(vec_x, vec_y) for vec_x, vec_y in zip(embeddings_x, embeddings_y)]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [61]:
true_labels = df['same_security']

threshold = 0.5
binary_preds_use = (df['cosine_similarity_use'] > threshold).astype(int)

# Calculate evaluation metrics for USE model
accuracy_use = accuracy_score(true_labels, binary_preds_use)
precision_use = precision_score(true_labels, binary_preds_use)
recall_use = recall_score(true_labels, binary_preds_use)
f1_use = f1_score(true_labels, binary_preds_use)
mse_use = mean_squared_error(true_labels, df['cosine_similarity_use'])

print("Universal Sentence Encoder (USE) Model Metrics:")
print(f"Accuracy: {accuracy_use:.2f}")
print(f"Precision: {precision_use:.2f}")
print(f"Recall: {recall_use:.2f}")
print(f"F1 Score: {f1_use:.2f}")
print(f"Mean Squared Error: {mse_use:.2f}")


Universal Sentence Encoder (USE) Model Metrics:
Accuracy: 0.76
Precision: 0.76
Recall: 0.99
F1 Score: 0.86
Mean Squared Error: 0.18


In [14]:
# Jaccard Similarity
def calculate_jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

df['jaccard_similarity'] = df.apply(lambda row: calculate_jaccard_similarity(set(word_tokenize(row['description_x'])), set(word_tokenize(row['description_y']))), axis=1)


In [62]:
true_labels = df['same_security']

threshold = 0.5
binary_preds_jaccard = (df['jaccard_similarity'] > threshold).astype(int)

# Calculate evaluation metrics for Jaccard Similarity
accuracy_jaccard = accuracy_score(true_labels, binary_preds_jaccard)
precision_jaccard = precision_score(true_labels, binary_preds_jaccard)
recall_jaccard = recall_score(true_labels, binary_preds_jaccard)
f1_jaccard = f1_score(true_labels, binary_preds_jaccard)
mse_jaccard = mean_squared_error(true_labels, df['jaccard_similarity'])

print("Jaccard Similarity Model Metrics:")
print(f"Accuracy: {accuracy_jaccard:.2f}")
print(f"Precision: {precision_jaccard:.2f}")
print(f"Recall: {recall_jaccard:.2f}")
print(f"F1 Score: {f1_jaccard:.2f}")
print(f"Mean Squared Error: {mse_jaccard:.2f}")


Jaccard Similarity Model Metrics:
Accuracy: 0.46
Precision: 0.89
Recall: 0.32
F1 Score: 0.47
Mean Squared Error: 0.30


In [15]:
# TF-IDF (Term Frequency-Inverse Document Frequency)
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['description_x'] + ' ' + df['description_y'])
tfidf_matrix_normalized = normalize(tfidf_matrix)
cosine_similarities_tfidf = cosine_similarity(tfidf_matrix_normalized)
df['cosine_similarity_tfidf'] = [cosine_similarities_tfidf[i, i] for i in range(len(df))]


In [63]:
true_labels = df['same_security']

threshold = 0.5
binary_preds_tfidf = (df['cosine_similarity_tfidf'] > threshold).astype(int)

# Calculate evaluation metrics for TF-IDF model
accuracy_tfidf = accuracy_score(true_labels, binary_preds_tfidf)
precision_tfidf = precision_score(true_labels, binary_preds_tfidf)
recall_tfidf = recall_score(true_labels, binary_preds_tfidf)
f1_tfidf = f1_score(true_labels, binary_preds_tfidf)
mse_tfidf = mean_squared_error(true_labels, df['cosine_similarity_tfidf'])

print("TF-IDF Model Metrics:")
print(f"Accuracy: {accuracy_tfidf:.2f}")
print(f"Precision: {precision_tfidf:.2f}")
print(f"Recall: {recall_tfidf:.2f}")
print(f"F1 Score: {f1_tfidf:.2f}")
print(f"Mean Squared Error: {mse_tfidf:.2f}")


TF-IDF Model Metrics:
Accuracy: 0.75
Precision: 0.75
Recall: 1.00
F1 Score: 0.86
Mean Squared Error: 0.25


In [21]:
print(df[['description_x', 'description_y', 'cosine_similarity_w2v', 'cosine_similarity_doc2vec', 'cosine_similarity_use', 'jaccard_similarity', 'cosine_similarity_tfidf']])

                                          description_x  \
0                        first trust dow jones internet   
1                   schwab intl large company index etf   
2                          vanguard small cap index adm   
3     duke energy corp new com new isin us4 sedol b7...   
4                                        visa inc class   
...                                                 ...   
2137                            delta air lines inc com   
2138                             cognizant tech solutns   
2139   pimco investment grade corporate bond fund class   
2140                                   eli lilly co com   
2141                                  dfa comm strategy   

                                          description_y  \
0                           first trust dj internet idx   
1     schwab strategic tr fundamental intl large co ...   
2                     vanguard smallcap index fund inst   
3     duke energy corp new com new isin us26441c2044...

In [24]:
predictions = []

In [25]:
model_names = []
model_names.append('Word2Vec')
model_names.append('Doc2Vec')
model_names.append('USE')
model_names.append('jaccard_similarity')
model_names.append('jaccard_similarity')

In [65]:
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
mse_scores = []

In [67]:
accuracy_scores = [accuracy_w2v,accuracy_doc2vec,accuracy_use,accuracy_jaccard,accuracy_tfidf]
precision_scores = [precision_w2v,precision_doc2vec,precision_use,precision_jaccard,precision_tfidf]
recall_scores = [recall_w2v,recall_doc2vec,recall_use,recall_jaccard,recall_tfidf]
f1_scores = [f1_w2v,f1_doc2vec,f1_use,f1_jaccard,f1_tfidf]
mse_scores = [mse_w2v,mse_doc2vec,mse_use,mse_jaccard,mse_tfidf]

In [70]:
summary_df = pd.DataFrame({
    'Model': model_names,
    'Accuracy': accuracy_scores,
    'Precision': precision_scores,
    'Recall': recall_scores,
    'F1 Score': f1_scores,
    'Mean Squared Error': mse_scores
})

In [71]:
summary_df

,Model,Accuracy,Precision,Recall,F1 Score,Mean Squared Error
0,Word2Vec,0.246965,0.000000,0.000000,0.000000,0.753035
1,Doc2Vec,0.755836,0.806524,0.889027,0.845768,0.345110
2,USE,0.758170,0.760095,0.991940,0.860678,0.177033
3,Jaccard Similarity,0.456583,0.893170,0.316181,0.467033,0.301214
4,Cosine Similarity TF-IDF,0.753035,0.753035,1.000000,0.859121,0.246965


In [72]:
summary_df.to_csv("summary_df.csv")